In [1]:
import json 
import h5py
import numpy as np

In [2]:
with h5py.File('SGG-Benchmark-main\datasets\VG150\mini_imdb_1024.h5','r') as f:
    print("Keys: %s" % f.keys())
    #a_group_key = list(f.keys())[0]
    h_keys=list(f.keys())

    data=f[h_keys[1]][:]

    

Keys: <KeysViewHDF5 ['image_heights', 'image_ids', 'image_widths', 'images', 'original_heights', 'original_widths', 'valid_idx']>


In [5]:
dict_data=json.load(open('mini-vg/mini_VG-SGG-dicts.json'))

In [11]:
import json 
object=json.load(open('objects.json'))

In [12]:
rel=json.load(open('relationships.json'))

In [14]:
def get_box_pos(x,y,orig_size=[6000,4000]):
    """
    inputs:
        bbox 框的位置 xywh
        orig_size 图像大小 x,y heng shu
    outputs:
        框所在图片位置
        左上角，上部，右上角
        左部，  中部，右部
        左下角  下部  右下角
        012
        345
        678
           
    """
    #center=((bbox[0]+bbox[2])/2,(bbox[1]+bbox[3])/2)    ##获取中心坐标
    #center=(bbox[0],bbox[1])
    center=[x,y]

    if center[0]<orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top left corner'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top'
    if center[0]>2*orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top right corner'
    if center[0]<orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'left'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'center'
    if center[0]>2*orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'right'
    if center[0]<orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower left corner'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower'
    if center[0]>2*orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower right corner'

In [4]:
img_data=json.load(open('image_data.json'))
rel=json.load(open('relationships.json'))

In [16]:
img_2_shape={}            #### 图像对应shape  用于判断方位
for i in img_data:
    if i['image_id'] in data:
        img_2_shape[i['image_id']]=[i['width'],i['height']]

In [18]:
label_150=json.load(open('labels.json'))
label_150_list=list(label_150.values())

In [19]:
bbox_list=[[[],[]] for _ in range(1000)]
id_list=[]
idx=0
for ob in object:
    if ob['image_id'] in data:
        id_list.append(ob['image_id'])
        for i in ob['objects']:
            if i['names'][0] not in label_150_list:
                continue
            bbox_list[idx][0].append(get_box_pos(i['x'],i['y'],img_2_shape[ob['image_id']]))      
            bbox_list[idx][1].append(i['names'][0])
        
        idx+=1

In [21]:
rel_list={}
for i in rel:
    if i['image_id'] in data:
        rel_list[i['image_id']]=i['relationships']

In [22]:
import os
import shutil
src_folder = 'image/VG_100K/'
# 目标文件夹路径
dst_folder = 'test_data/'

for id in id_list:
    shutil.copy2(src_folder+str(id)+'.jpg', dst_folder)



In [23]:
def get_ground_truth(bbox_list,id_list,rel_list,label_150_list):
    """生成ground_truth"""
    ground_list={}

    for k,single_bbox in enumerate(bbox_list):        ##便利每一张图的box
        ground={
            "number":{},
            "location":{},
        }

        count_dict={}                          ###数量
        for item in single_bbox[1]:
            if item in count_dict:
                count_dict[item] += 1
            else:
                count_dict[item] = 1

        keys=count_dict.keys()                   ### 位置
        loc_dict={key: [] for key in keys}

        for i in range(len(single_bbox[0])):
            loc_dict[single_bbox[1][i]].append(single_bbox[0][i])

        ###
        r_list=[]
        for r in rel_list[id_list[k]]:                ###便利关系
            if r['object']['name'] not in label_150_list or r['subject']['name'] not in label_150_list:
                continue
            r_list.append([r['object']['name'],r['predicate'].lower(),r['subject']['name']])


        ground['number']=count_dict
        ground['location']=loc_dict
        ground['relationship']=r_list

        ground_list[id_list[k]]=ground

    return ground_list

In [24]:
ground_list=get_ground_truth(bbox_list,id_list,rel_list,label_150_list)

In [25]:
import pickle 
with open('VG_ground_truth.pickle','wb') as f:
    pickle.dump(ground_list,f)